This notebook gets, for each user, the match history of that user.

In [ ]:
import requests
import pandas as pd
import os
import glob
import time
import math
import random

In [ ]:
api_key = "YOUR API KEY HERE"

In [ ]:
failed_account_ids = []

In [ ]:
if not os.path.exists('data/failed_match_history_dir'):
    os.mkdir('data/failed_match_history_dir')

In [ ]:
#The team that runs the match service uses account IDs, and the team that runs the leagues service uses summoner IDs. 
#summonerid -> accountId/id
def get_match_history(accountId):
    jsons = []
    begin_index = 0
    while True:
        time.sleep(3)
        params = {'api_key':api_key, 'beginIndex':begin_index}
        print('https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?'.format(accountId))
        print(params)
        r = requests.get('https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?'.format(accountId), params = params)
        print(r.url)
        if r.status_code == 403:
            print('sleeping for an hour')
            time.sleep(3600)
        if r.status_code > 400:
            print(r.status_code)
            time.sleep(120)
            r = requests.get('https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?'.format(accountId), params = params)
            if r.status_code > 400:
                print('failed')
                failed_account_ids.append(accountId)
                return False
        #we've gotten all the matches already
        if len(r.json()['matches']) == 0:
            break
        jsons += r.json()['matches']
        begin_index += 100
    return(jsons)

In [ ]:
pages = glob.glob('data/pages_with_id/*.csv')

In [ ]:
if not os.path.isdir('data/match_history_plat'):
    os.mkdir('data/match_history_plat')
if not os.path.isdir('data/match_history'):
    os.mkdir('data/match_history')

In [ ]:
random_number = random.randint(0, 100000)
for p in pages:
    page_num = int(os.path.basename(p).split('.')[0].split('_')[1])
    dir_str = 'data/match_history'
    if page_num >= 100:
        dir_str += '_plat'
    already_scraped = glob.glob(dir_str+'/*.csv')
    page = pd.read_csv(p)
    account_ids = list(page['account_id'])
    for ai in account_ids:
        ai = str(ai)
        if ai == 'nan':
            continue
        #check if we already did ai
        if os.path.join(dir_str, str(ai)+'.csv') in already_scraped:
            continue
        #if not, get the match history
        foo = get_match_history(ai)
        if foo:
            df = pd.DataFrame.from_dict(foo)
            df.to_csv(dir_str+'/{}.csv'.format(str(ai)))
        failed = pd.DataFrame({'failed_accountId':failed_account_ids})
        failed.to_csv('data/failed_match_history_dir/failed_match_history_scrapes_'+str(random_number)+'.csv')